In [ ]:
from skimage.feature import greycomatrix,greycoprops
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import os
import numpy as np
from PIL import Image
from skimage.color import rgb2gray
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
!unzip FMD.zip

In [ ]:
from skimage import feature
def GLCM(img):
  feature_vector=[]
  img = rgb2gray(img)
  img = img.astype(np.uint8)
  glcm = greycomatrix(img, [1], [0], 256, symmetric=True, normed=True)
  feature_vector.append(greycoprops(glcm, 'contrast')[0,0])
  feature_vector.append(greycoprops(glcm, 'homogeneity')[0, 0])
  feature_vector.append(greycoprops(glcm, 'energy')[0, 0])
  feature_vector.append(greycoprops(glcm, 'correlation')[0, 0])
  feature_vector = np.array(feature_vector)
  return feature_vector

def getFeatureVector(images):
  features_for_one_category=[]
  features_for_all_categories=[]
  for i in range(images.shape[0]):
    for j in range(images.shape[1]):
      fv=GLCM(images[i][j])
      features_for_one_category.append(fv)
    features_for_all_categories.append(features_for_one_category)
    features_for_one_category=[]
  features_for_all_categories = np.array(features_for_all_categories)
  return features_for_all_categories

def distance(test,train):
  return np.sqrt(np.sum(np.square(test-train)))

# x is a test feature vector
# y in a train feature vector
def KNN(test,train):
  mini=1e9
  index=-1
  for i in range(train.shape[0]): # train.shape[0]=10
    for j in range(train.shape[1]): # train.shape[1]=70
      d=distance(test,train[i][j]) # train[0][0] = 4
      print(d)
      if mini>d:
        index = i
      mini = min(mini,d)
  return index

def accuracy(xtrain,xtest,dirnames):
  error=[]
  counter=0
  for i in range(xtest.shape[0]): # xtext.shape[0]=10
    for k in range(xtest.shape[1]): #xtest.shape[1]=30
      j = KNN(xtest[i][k],xtrain)
      if dirnames[j]==dirnames[i]:
        counter+=1
      else:
        error.append(i)
  error = np.array(error)
  return (counter/300)*100

def convert_image2array(dir):
  category = []
  matrix=[] 
  directory = dir
  dirnames=[]
  for root, dirs, files in os.walk(directory):
    if len(files)==0:
      continue
    for filename in files:
      img = load_img(os.path.join(root,filename))
      data = img_to_array(img)
      matrix.append(data)
    category.append(matrix)
    matrix=[]
    dirnames.append(os.path.split(root)[-1])
  category = np.array(category)
  return category,dirnames

In [ ]:
images,dirs=convert_image2array('/content/image')
print(dirs)

['fabric', 'stone', 'metal', 'water', 'foliage', 'glass', 'wood', 'paper', 'leather', 'plastic']


In [ ]:
images.shape

(10, 100, 384, 512, 3)

In [ ]:
features = getFeatureVector(images)  

In [ ]:
features.shape

(10, 100, 4)

In [ ]:
train = features[:,:70,:,]
test = features[:,70:,:]
print(train[0][0].shape,test.shape)
print(test.shape)
print(test[0].shape)
print(train[0].shape)
print(train[0][0])
print(test[0][0])

(4,) (10, 30, 4)
(10, 30, 4)
(30, 4)
(70, 4)
[1.31890992e+02 2.52705622e-01 3.88537172e-02 9.77677948e-01]
[1.78951271e+03 3.83990085e-02 7.61154365e-03 7.05462830e-01]


In [ ]:
ac  = accuracy(train,test,dirs)
print("the accuracy is " ,ac,"%" )